# 20231213_FED_DOT_PLOT

https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20231213.htm

In [2]:
import pandas as pd
import bs4
import requests
import numpy
import plotly.express as px

def request_dots(path='https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20231213.htm', spacing=30, target_table=21):
    # Download file
    res = requests.get(path)
    # Check for errors
    try:
        res.raise_for_status()
    except Exception as exc:
        print('There was a problem: %s' % (exc))
    
    # Save file to disk
    projectionFile = open('fomcprojtabl.htm', 'wb')
    for chunk in res.iter_content(100000):
        projectionFile.write(chunk)
    projectionFile.close()
    
    # Make the soup
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    # Find the public tables
    tables = soup.select('table[class="pubtables"]')
    
    # Parse the target table only    
    df = parse_table(tables[-target_table])
    # Expand count of participants to an array to identify each dot individually
    dfsize = df.shape
    for rows in range(0, dfsize[0]-1):
        for cols in range (0, dfsize[1]):
            count = df.iloc[rows][cols]
            if count:
                array = range(1, int(count)+1)
            else:
                array = [0]
            df.iloc[rows][cols] = array
    
    # Restrict the dataframe to 4 columns (Current, 1Yr, 2Yr, Longer-term)
    if(len(df.columns) == 5):
        df = df.iloc[:,[0,1,2,3,4]]
    df['Projection'] = df.index
    str(df.columns[0:5])
    df.columns = ['2023', '2024', '2025', '2026', 'Longer Run','Projection']
    dl = pd.melt(df,  id_vars='Projection', value_vars=['2023', '2024', '2025', '2026', 'Longer Run'])      
    dl['year'] = dl['variable']
    
    dl = dl.copy() # 경고삭제
    dl.loc[dl['variable'] == 'Longer Run', 'year'] = int(list(df.columns)[1])+3 
    
    xvalue_res = [] 
    yvalue_res = [] 
    dl = dl[dl['value'] !="" ]
    dl['year'] = dl['year'].astype(float)
    dl['Projection'] = dl['Projection'].astype(float)
    
    # Spread the dots on the x-axis depending on how many there are     
    for rows in range(0, len(dl['value'])):    
        xvalue = numpy.repeat(dl['year'].iloc[rows], dl['value'].iloc[rows], axis=None)
        xvalue = xvalue.astype(int)
        yvalue = numpy.repeat(dl['Projection'].iloc[rows], dl['value'].iloc[rows], axis=None)
        if int(dl['value'].iloc[rows]) == 1:
            xvalue = numpy.linspace(int(dl['year'].iloc[rows]) - 0, int(dl['year'].iloc[rows]) + 0, int(dl['value'].iloc[rows]))
        else:
            xvalue = numpy.linspace(int(dl['year'].iloc[rows]) - int(dl['value'].iloc[rows]) / spacing, int(dl['year'].iloc[rows]) + int(dl['value'].iloc[rows]) / spacing, int(dl['value'].iloc[rows]))
        xvalue_res = numpy.concatenate([xvalue_res, xvalue])
        yvalue_res = numpy.concatenate([yvalue_res, yvalue])
    
    return dl, xvalue_res, yvalue_res

# Parse table function
def parse_table(table):
    # Parse rows
    bdata = []
    rows = table.find_all('tr')
    for row in rows:
        # find first column header
        cols0 = row.find_all('th')
        cols0 = [ele.text.strip() for ele in cols0]
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        cols = [ele for ele in cols0] + [ele for ele in cols]
        bdata.append(cols)
    # Convert to DataFrame
    bdata[0][0] = "MidpointTargetRange" 
    bdata[0][-1]= "Longer Run"
    df = pd.DataFrame(bdata[1:], columns=bdata[0])
    df.set_index("MidpointTargetRange", inplace=True)
    return df

# request_dots 함수를 호출하고 반환된 값을 받음
dl, xvalue_res, yvalue_res = request_dots()

# Plotting code
fig = px.scatter(x=xvalue_res, y=yvalue_res, title='FOMC 점도표(2023년 12월 13일 발표)', labels={"x": "", "y": ""}, template='plotly_white',
                 range_x=[2022, 2028], range_y=[0, 8])

# X 축 라벨 값 수정
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [2022, 2023, 2024, 2025, 2026, 2027, 2028],
        ticktext = ['2022', '2023', '2024', '2025', '2026', 'LONG RUN', '']
    )
)

# 2026과 LONG RUN 사이에 수직선 추가
fig.add_shape(
    dict(
        type="line",
        x0=2026.5,
        y0=0,
        x1=2026.5,
        y1=8,
        line=dict(
            color="Red",
            width=1,
            dash="dashdot",
        )
    )
)

# Y 축 TICK을 0.25마다로 변경
fig.update_layout(
    yaxis = dict(
        tick0 = 0,
        dtick = 0.25
    )
)

# 그리드를 점선으로 추가
fig.update_layout(
    xaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgrey',
        zeroline=True,
    ),
    yaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgrey',
        zeroline=True,
    )
)
fig.show()

config = {
    'toImageButtonOptions': {
        'format': 'svg',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 1000,
        'scale': 1  # Multiply title/legend/axis/canvas sizes by this factor
    }
}
fig.write_html("2024_StaticDotplot.html", config=config)


<ipython-input-2-d8664fd4ec59>:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [3]:
import pandas as pd  # pandas 라이브러리를 pd로 가져오기
import bs4  # BeautifulSoup 라이브러리 가져오기
import requests  # requests 라이브러리 가져오기
import numpy  # numpy 라이브러리 가져오기
import plotly.express as px  # plotly.express 라이브러리를 px로 가져오기

def request_dots(path='https://www.federalreserve.gov/monetarypolicy/fomcprojtabl20231213.htm', spacing=30, target_table=21):
    # 파일 다운로드
    res = requests.get(path)
    # 오류 확인
    try:
        res.raise_for_status()
    except Exception as exc:
        print('문제 발생: %s' % (exc))
    
    # 파일 디스크에 저장
    projectionFile = open('fomcprojtabl.htm', 'wb')
    for chunk in res.iter_content(100000):
        projectionFile.write(chunk)
    projectionFile.close()
    
    # BeautifulSoup을 사용하여 파싱
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    # 공개 테이블 찾기
    tables = soup.select('table[class="pubtables"]')
    
    # 목표 테이블만 파싱    
    df = parse_table(tables[-target_table])
    # 참가자 수를 배열로 확장하여 각 점을 식별
    dfsize = df.shape
    for rows in range(0, dfsize[0]-1):
        for cols in range (0, dfsize[1]):
            count = df.iloc[rows][cols]
            if count:
                array = range(1, int(count)+1)
            else:
                array = [0]
            df.iloc[rows][cols] = array
    
    # 데이터프레임을 4개 열로 제한 (현재, 1년, 2년, 장기)
    if(len(df.columns) == 5):
        df = df.iloc[:,[0,1,2,3,4]]
    df['Projection'] = df.index
    str(df.columns[0:5])
    df.columns = ['2023', '2024', '2025', '2026', 'Longer Run','Projection']
    dl = pd.melt(df,  id_vars='Projection', value_vars=['2023', '2024', '2025', '2026', 'Longer Run'])      
    dl['year'] = dl['variable']
    
    dl = dl.copy() # 경고제거
    dl.loc[dl['variable'] == 'Longer Run', 'year'] = int(list(df.columns)[1])+3 
    
    xvalue_res = [] 
    yvalue_res = [] 
    dl = dl[dl['value'] !="" ]
    dl['year'] = dl['year'].astype(float)
    dl['Projection'] = dl['Projection'].astype(float)
    
    # x축에 따라 점을 분산
    for rows in range(0, len(dl['value'])):    
        xvalue = numpy.repeat(dl['year'].iloc[rows], dl['value'].iloc[rows], axis=None)
        xvalue = xvalue.astype(int)
        yvalue = numpy.repeat(dl['Projection'].iloc[rows], dl['value'].iloc[rows], axis=None)
        if int(dl['value'].iloc[rows]) == 1:
            xvalue = numpy.linspace(int(dl['year'].iloc[rows]) - 0, int(dl['year'].iloc[rows]) + 0, int(dl['value'].iloc[rows]))
        else:
            xvalue = numpy.linspace(int(dl['year'].iloc[rows]) - int(dl['value'].iloc[rows]) / spacing, int(dl['year'].iloc[rows]) + int(dl['value'].iloc[rows]) / spacing, int(dl['value'].iloc[rows]))
        xvalue_res = numpy.concatenate([xvalue_res, xvalue])
        yvalue_res = numpy.concatenate([yvalue_res, yvalue])
    
    return dl, xvalue_res, yvalue_res

# 테이블 파싱 함수
def parse_table(table):
    # 행 파싱
    bdata = []
    rows = table.find_all('tr')
    for row in rows:
        # 첫번째 열 헤더 찾기
        cols0 = row.find_all('th')
        cols0 = [ele.text.strip() for ele in cols0]
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        cols = [ele for ele in cols0] + [ele for ele in cols]
        bdata.append(cols)
    # DataFrame으로 변환
    bdata[0][0] = "MidpointTargetRange" 
    bdata[0][-1]= "Longer Run"
    df = pd.DataFrame(bdata[1:], columns=bdata[0])
    df.set_index("MidpointTargetRange", inplace=True)
    return df

# request_dots 함수 호출 및 반환값 받기
dl, xvalue_res, yvalue_res = request_dots()

# 그래픽 코드
fig = px.scatter(x=xvalue_res, y=yvalue_res, title='FOMC 점도표', labels={"x": "", "y": ""}, template='plotly_white',
                 range_x=[2022, 2028], range_y=[0, 8])

# X 축 라벨 값 수정
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [2022, 2023, 2024, 2025, 2026, 2027, 2028],
        ticktext = ['2022', '2023', '2024', '2025', '2026', 'LONG RUN', '']
    )
)

# 2026과 LONG RUN 사이에 수직선 추가
fig.add_shape(
    dict(
        type="line",
        x0=2026.5,
        y0=0,
        x1=2026.5,
        y1=8,
        line=dict(
            color="Red",
            width=1,
            dash="dashdot",
        )
    )
)

# Y 축 TICK을 0.25마다로 변경
fig.update_layout(
    yaxis = dict(
        tick0 = 0,
        dtick = 0.25
    )
)

# 그리드를 점선으로 추가
fig.update_layout(
    xaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgrey',
        zeroline=True,
    ),
    yaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgrey',
        zeroline=True,
    )
)
fig.show()

config = {
    'toImageButtonOptions': {
        'format': 'svg',  # one of png, svg, jpeg, webp
        'filename': 'custom_image',
        'height': 500,
        'width': 1000,
        'scale': 1  # Multiply title/legend/axis/canvas sizes by this factor
    }
}
fig.write_html("2024_StaticDotplot.html", config=config)
# 기존 코드
fig.show()

config = {
    'toImageButtonOptions': {
        'format': 'svg',  # png, svg, jpeg, webp 중 하나
        'filename': 'custom_image',
        'height': 500,
        'width': 1000,
        'scale': 1  # 제목/범례/축/캔버스 크기에 이 배수 적용
    }
}
fig.write_html("2024_StaticDotplot.html", config=config)


<ipython-input-3-40a3c9d4a284>:38: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

